# Some interesting charts for Covid-19 in England

In [ ]:
import os
import datetime
fetch_date = os.environ.get('TODAY', 'test')
fetch_time = os.environ.get('FETCH_TIME', 'a test run')

In [ ]:
input_filename = 'covid_{}.csv'.format(fetch_date)

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
display(HTML('<p>Data was fetched from coronavirus.data.gov.uk on {}.</p>'.format(fetch_time)))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [ ]:
df = pd.read_csv(input_filename).set_index('date').sort_index()
df.index = pd.to_datetime(df.index)

In [ ]:
def plot_chart(df, title):
    ax = df.plot.line(title=title, figsize=(14,8))
    ax.grid(True, which='both')
    plt.xticks(rotation=45)
    ax.set_xlim(xmin=df.index[0])
    ax.set_ylim(ymin=0)
    plt.show()

In [ ]:
df_hospital = df[['hospitalCases', 'covidOccupiedMVBeds', 'newDeathsByDeathDate']].dropna(how='all')
plot_chart(df_hospital, 'Cases in hospital; occupied ventilators; deaths: all available data')

In [ ]:
df_deaths = df[['newDeathsByDeathDate']].dropna()
plot_chart(df_deaths, 'Deaths all available data')

In [ ]:
df_hospital_last_60 = df[['hospitalCases', 'covidOccupiedMVBeds', 'newDeathsByDeathDate']].dropna(how='all').tail(60)
plot_chart(df_hospital_last_60, 'Cases in hospital; occupied ventilators; deaths: over last 60 days')


In [ ]:
df['deaths 7 day mean'] = df['newDeathsByDeathDate'].rolling(7).mean()
df_deaths_last_60 = df[['newDeathsByDeathDate', 'deaths 7 day mean']].dropna().tail(60)
plot_chart(df_deaths_last_60, 'Deaths over last 60 days')

In [ ]:
df['occupied ventilators 7 day mean'] = df['covidOccupiedMVBeds'].rolling(7).mean()
df_ventilators_last_60 = df[['covidOccupiedMVBeds', 'occupied ventilators 7 day mean']].dropna().tail(60)
plot_chart(df_ventilators_last_60, 'Occupied ventilators over last 60 days')

In [ ]:
df['cases 7 day mean'] = df['newCasesByPublishDate'].rolling(7).mean()
df_cases_last_60 = df[['newCasesByPublishDate', 'cases 7 day mean']].dropna().tail(60)
plot_chart(df_cases_last_60, 'Cases over last 60 days')

In [ ]:
df['total tests'] = df.apply(lambda x: x['newPillarOneTestsByPublishDate'] + x['newPillarTwoTestsByPublishDate'], axis=1)
df['total tests 7 day mean'] = df[['total tests']].rolling(7).mean()
df_test_last_60 = df[['newPillarOneTestsByPublishDate', 'newPillarTwoTestsByPublishDate', 'total tests', 'total tests 7 day mean']].dropna(how='all').tail(60)
plot_chart(df_test_last_60, 'Tests over last 60 days')

In [ ]:
df['percentage cases per test'] = df.apply(lambda x: x['newCasesByPublishDate'] * 100.0 / x['total tests'], axis=1)
df['7 day mean percentage cases per test'] = df['percentage cases per test'].rolling(7).mean()
df_cases_per_test_last_60 = df[['percentage cases per test', '7 day mean percentage cases per test']].dropna().tail(60)
plot_chart(df_cases_per_test_last_60, '% Cases per test over last 60 days')

In [ ]:
df_cases_per_test_all = df[['percentage cases per test', '7 day mean percentage cases per test']].dropna()
plot_chart(df_cases_per_test_all, 'Cases per test all data')

In [ ]:
df.to_csv('covid-processed-{}.csv'.format(fetch_date))

In [ ]:
display(HTML('<p>Data in these charts: <a href="covid-processed-{}.csv">covid-processed-{}.csv</a></p>'.format(
    fetch_date, fetch_date)))
display(HTML('<p><a href="https://github.com/RobFisher/rfcovid">Source code</a>.</p>'))
